# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Setup

Import the json file into MongoDB using the following command

`mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [22]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [23]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [24]:
# Confirm that the new database was created
print(mongo.list_database_names())

['admin', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'uk_food']


In [25]:
# Assign the uk_food database to a variable name
db = mongo['uk_food']

In [26]:
# Review the collections in new database
print(db.list_collection_names())

['establishments']


In [27]:
# Review a document in the establishment collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fa58f73fad540eb1b4fc29'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [28]:
# Assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked it to be included it the analysis. Add the restaurant "Penang Flavours" to the database and update its business type ID.

In [29]:
# Insert a new document into the collection
penang_flavours ={
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}
establishments.insert_one(penang_flavours)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fa58fe4df1b079b443f14a'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [37]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields.
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"_id": 0, "BusinessType": 1, "BusinessTypeID": 1}

# Capture the results to a variable
results = establishments.find_one(query, fields)

# Prerry Print the result
pprint(results)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


In [39]:
# Update the new restaurant with the BusinessTypeID found above
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set":
        {"BusinessTypeID": 1}
     }
)
# Confirm that the restaurant was updated
establishments.find_one({"BusinessName": "Penang Flavours"})

{'_id': ObjectId('64fa58fe4df1b079b443f14a'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

The magazine is not interested in any establishments in Dover. Check how many documents contain the Dover Local Authority, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.